In [1]:
import findspark
findspark.init('C:\spark-3.0.0-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('mylogproject').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
train_data=spark.read.csv('hr_data.csv', inferSchema=True,header=True)

In [4]:
train_data.groupBy('left').count().show()

+----+-----+
|left|count|
+----+-----+
|   1| 3571|
|   0|11428|
+----+-----+



In [5]:
train_data = train_data.dropna()

In [6]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline

department_indexer=StringIndexer(inputCol='department',outputCol='departmentIndex')
department_enc=OneHotEncoder(inputCol='departmentIndex',outputCol='departmentVecor')

salary_indexer=StringIndexer(inputCol='salary',outputCol='salaryIndex')
salary_enc=OneHotEncoder(inputCol='salaryIndex',outputCol='salaryVecor')

In [7]:
assembler=VectorAssembler(inputCols=['number_project','average_montly_hours','time_spend_company',
                                     'Work_accident','promotion_last_5years','departmentVecor','salaryVecor'], 
                          outputCol='features')
log_reg_hr=LogisticRegression(featuresCol='features',labelCol='left')

In [8]:
pipeline=Pipeline(stages=[department_indexer,salary_indexer,
                 department_enc,salary_enc,
                 assembler,log_reg_hr])

In [9]:
train_data,test_data=train_data.randomSplit([0.7,0.3])

In [16]:
#test_data.show(3)

In [17]:
fit_model=pipeline.fit(train_data)
final_result= fit_model.transform(test_data)

In [18]:
#final_result.filter(final_result['employee_id']==1038).head(1)

In [19]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval=BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='left')
evalution_result=my_eval.evaluate(final_result)
evalution_result

0.5067859586015193

In [20]:
#final_result.printSchema()

In [21]:
#final_result.select('employee_id','left','prediction').show(100)